In [13]:
from nltk.corpus import sentiwordnet as swn
from nltk import pos_tag
import json
import sys
import tqdm

In [2]:
# Deprecated. See get_score()
def get_scores(word, postag):
    """  Function to get sentiment scores from SentiWordNet.
    Args:
        word: word to get sentiment scores
        postag: pos tag of the word
    Returns:
        score: a senti_synset instance that has negative, positive and neutral
        scores of the word"""

    if postag == "Noun":  # NOUN
        score = swn.senti_synset(word + '.n.01')
        return score
    elif postag == "Verb":  # VERB
        score = swn.senti_synset(word + '.v.01')
        return score
    elif postag == "Adjective":  # ADJECTIVE
        score = swn.senti_synset(word + '.a.01')
        return score
    elif postag == "Adverb":  # ADVERB
        score = swn.senti_synset(word + '.r.01')
    return score

In [14]:
# Read Data
with open('./neg-20.json') as data_file:
    data = data_file.readlines()

data = [x.strip() for x in data]
jsonSentences = [json.loads(x) for x in data]

In [14]:
# Row 6217, there is only posTag!!!
for i in range(len(jsonSentences[6216])):
    try:
        print(jsonSentences[6216][i]["word"])
    except Exception as e:
        print(i+1)

unk
age
formula
cat
attractive
come
veterinary
side
strongly
to
be
taken
direction
warn
brand
minnoş
blackout
veterinary
give
scolding
hear
in
the
future
bowel
pee
way
28
problem
way
flow
say
order
make
veterinary
consult
say


In [18]:
''' All words '''
sentiments = []
for idx, sentence in enumerate(tqdm.tqdm(jsonSentences)):
    total = 0.0
    for i in range(len(sentence)):
        word = sentence[i]
        flag = 0
        new_tag = ""
        if (word["word"] == "unk"):
            continue
        try:
            if (i != (len(sentence) -1) and sentence[i+1]["word"] == "not"):
                flag = 1
            elif (i != 0 and sentence[i-1]["word"] == "bad"):
                flag = 1
            scores = get_score(word["word"], word["posTag"], flag)
            if (not scores):
                new_tag = eng_tag(word["word"])
#                 print(word["word"] + " " + new_tag)
                scores = get_score(word["word"], new_tag, flag)
            score = calculate_one_score(scores[:3])
        except Exception as e:
            print(e)
#             print(idx)
            continue
        total += score
    sentiments.append(total)
# for i in range(len(sentiments)):
#     print("{0}. Sentence: {1:.3f}".format(i+1, sentiments[i]))

100%|██████████| 20/20 [00:00<00:00, 588.37it/s]


In [19]:
print(sentiments)

[-0.5928030303030303, -0.4172979797979798, 0.0625, -0.8327020202020203, -0.332070707070707, 0.011363636363636402, -0.5643939393939394, 0.0, -0.1268939393939394, -0.4943181818181818, -0.4943181818181818, -0.7506313131313131, -0.5416666666666667, -0.08522727272727273, -0.062499999999999986, -0.5284090909090909, 0.00568181818181818, -0.7335858585858586, -0.4431818181818182, 0.5132575757575758]


In [7]:
''' Only adjectives and noun '''
sentiments = []
l = len(jsonSentences)
for idx, sentence in enumerate(tqdm.tqdm(jsonSentences)):
    total = 0.0
    n_sent = ""
    for i in range(len(sentence)):
        word = sentence[i]
        flag = 0
        new_tag = ""
        if (word["posTag"] == "Adjective"):
            try:
                if (i != (len(sentence) -1) and sentence[i+1]["word"] == "not"):
                    flag = 1
                elif (i != 0 and sentence[i-1]["word"] == "bad"):
                    flag = 1
                scores = get_score(word["word"], word["posTag"], flag)
                if (not scores):
                    new_tag = eng_tag(word["word"])
                    scores = get_score(word["word"], new_tag, flag)
                score = calculate_one_score(scores[:3])
            except Exception as e:
                continue
        elif (i != 0 and word["posTag"] == "Noun" and sentence[i-1]["posTag"] == "Adjective"):
            try:
                if (i != (len(sentence) -1) and sentence[i+1]["word"] == "not"):
                    flag = 1
                elif (i != 0 and sentence[i-1]["word"] == "bad"):
                    flag = 1
                scores = get_score(word["word"], word["posTag"], flag)
                if (not scores):
                    new_tag = eng_tag(word["word"])
                    scores = get_score(word["word"], new_tag, flag)
                score = calculate_one_score(scores[:3])
            except Exception as e:
                continue
        else:
            continue
        n_sent += str(word["word"] + " ").lower()
        total += score
    sentiments.append(total)
# for i in range(len(sentiments)):
#     print("{0}. Sentence: {1:.3f}".format(i+1, sentiments[i]))

100%|██████████| 14881/14881 [00:27<00:00, 540.72it/s]


In [9]:
true_preds = sum(1 for x in sentiments if x < 0)
total = len(sentiments)
print(str(true_preds)+"/"+str(total))

15/20


In [21]:
false_positive = {}
for i in range(len(sentiments)):
    if sentiments[i] > 0:
        false_positive = {i, sentiments[i]}

In [11]:
print(false_positive[:20])

[2, 5, 16, 19, 20, 21, 24, 27, 28, 30, 31, 32, 33, 34, 36, 37, 40, 42, 44, 46]


In [18]:
with open("false_pos.txt", 'w') as falses:
    falses.writelines(str(false_positive))

In [22]:
len(false_positive)

2

In [23]:
false_positive

{0.9955808080808082, 14877}

In [3]:
def get_score(word, postag, next_not):
    scores = []
    counter = 1
    true_score = 0
    while True:
        try:
            if postag == "Noun":  # NOUN
                score = swn.senti_synset(word + '.n.0' + str(counter))
            elif postag == "Verb":  # VERB
                score = swn.senti_synset(word + '.v.0' + str(counter))
            elif postag == "Adjective":  # ADJECTIVE
                score = swn.senti_synset(word + '.a.0' + str(counter))
            elif postag == "Adverb":  # ADVERB
                score = swn.senti_synset(word + '.r.0' + str(counter))

            true_score = score.pos_score() - score.neg_score()
            if (next_not == 1 and true_score > 0):
                true_score = true_score * (-1)
            scores.append(true_score)
            counter += 1
        except Exception as e:
#                 print(e)
                break
    return scores

In [4]:
def calculate_one_score(scores):
    if len(scores) == 0:
        return 0.0
    
    numerator = 0.0
    denominator = 0.0
    for i in range(len(scores)):
        numerator += scores[i] * (1 / (i+2))
        denominator += 1 / (i+1)
    return numerator / denominator
    

In [5]:
def eng_tag(word):
    lst = [word]
    raw = pos_tag(lst)
    eng_pos = raw[0][1]
    if eng_pos == "NN":
        return "Noun"
    elif eng_pos == "VB":
        return "Verb"
    elif eng_pos == "RB":
        return "Adverb"
    elif eng_pos == "JJ":
        return "Adjective"
    else:
        return None